In [1]:
import pandas as pd
import numpy as np
import random
import re
from tqdm import tqdm

# **Load ABCD Airline File**

In [2]:
df = pd.read_parquet("../data/ABCD_tripfiles.parquet", engine="pyarrow")
df.head()

,id,creation_time,airline_code,flight_number,flight_date,departure_airport,user_name,action_name,header_line,entry_details
0,33902381,2024-05-01 09:14:42,AB,2359,1,BLR,service-acco,ASMMsgProcessor,"2024-05-01 09:14:42,420 INFO [3f326d134d9de45...","<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<ns2..."
1,33910427,2024-05-01 10:28:12,AB,2104,2,BLR,service-acco,ASMMsgProcessor,"2024-05-01 10:28:12,745 INFO [de24be75947007f...","<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<ns2..."
2,33910380,2024-05-01 10:28:11,AB,2109,2,BLR,service-acco,ASMMsgProcessor,"2024-05-01 10:28:11,301 INFO [df3a1cc7d574161...","<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<ns2..."
3,33909790,2024-05-01 10:28:12,AB,2110,2,BOM,service-acco,ASMMsgProcessor,"2024-05-01 10:28:12,189 INFO [9e66dfb1ffe9048...","<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<ns2..."
4,33902706,2024-05-01 09:14:41,AB,2127,2,BOM,service-acco,ASMMsgProcessor,"2024-05-01 09:14:41,884 INFO [829566f527b588f...","<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<ns2..."


In [3]:
df.columns

Index(['id', 'creation_time', 'airline_code', 'flight_number', 'flight_date',
       'departure_airport', 'user_name', 'action_name', 'header_line',
       'entry_details'],
      dtype='object')

# **Actions**

In [4]:
unique_actions = df.action_name.unique()
len(unique_actions)

49

In [5]:
unique_actions

array(['ASMMsgProcessor', 'AcceptTransitDataAction',
       'AcceptTransitLoadAction', 'AcceptTransitNotocAction',
       'AssignLCCAction', 'AssignLoadplanAction',
       'AssignUnassignViewAction', 'CalculateWeightAndTrimAction',
       'CargoFinalAction', 'ChangeFlightLegStateAction',
       'CheckinMsgProcessor', 'CloseLegAction',
       'CreatePostDepartureMessagesAction',
       'CreateLoadingInstructionAction', 'CreateLoadsheetAction',
       'CreateZFWMessageAction', 'CreateBaggageLoadItemsAction',
       'ClearFlightsAction', 'InternalCreateLoadingInstructionAct',
       'InternalCreateLoadsheetAction', 'FlightPlanFiguresInMsgProcessor',
       'RampFinalAction', 'CrewMsgProcessor', 'PAXBOOKINGINMsgProcessor',
       'EstimateStorePaxDataAction', 'ReopenLegAction',
       'GetCabinConfigurationsAction', 'FuelDataInitializer',
       'SendPostDepartureMessagesAction',
       'SetActualBagWeightIndicatorAction', 'TransferCheckinDataAction',
       'StoreAircraftDataAction', 'Spe

In [6]:
df.action_name.value_counts()

action_name
CalculateWeightAndTrimAction           231990
StorePaxDataAction                     150871
CheckinMsgProcessor                    133782
UpdateLoadTableAction                   23554
ASMMsgProcessor                         22438
AssignLCCAction                         18162
UpdateFlightAction                       9468
PAXBOOKINGINMsgProcessor                 8878
ChangeFlightLegStateAction               8836
StoreRegistrationAndConfigurationAc      8036
UpdateSupplementaryInfoAction            7936
UpdateCrewDataAction                     7173
TransferCheckinDataAction                6586
CreateLoadingInstructionAction           6408
CrewMsgProcessor                         5444
EstimateStorePaxDataAction               5069
CreateLoadsheetAction                    5055
CreateZFWMessageAction                   5005
RampFinalAction                          3804
UpdateFuelDataAction                     3583
CreatePostDepartureMessagesAction        3406
AssignUnassignViewActi

In [7]:
df.user_name.value_counts()

user_name
service-acco    574407
human           135206
Name: count, dtype: int64

# **Parse Entry Details**

# Standard Inputs

In [8]:
standard_df = df[['id', 'creation_time', 'airline_code', 'flight_number', 'flight_date',
       'departure_airport', 'user_name', 'action_name']]

standard_df

,id,creation_time,airline_code,flight_number,flight_date,departure_airport,user_name,action_name
0,33902381,2024-05-01 09:14:42,AB,2359,1,BLR,service-acco,ASMMsgProcessor
1,33910427,2024-05-01 10:28:12,AB,2104,2,BLR,service-acco,ASMMsgProcessor
2,33910380,2024-05-01 10:28:11,AB,2109,2,BLR,service-acco,ASMMsgProcessor
3,33909790,2024-05-01 10:28:12,AB,2110,2,BOM,service-acco,ASMMsgProcessor
4,33902706,2024-05-01 09:14:41,AB,2127,2,BOM,service-acco,ASMMsgProcessor
...,...,...,...,...,...,...,...,...
709608,33875192,2024-05-01 04:07:01,AB,2372,30,GAU,service-acco,StorePaxDataAction
709609,33875261,2024-05-01 04:08:01,AB,2372,30,GAU,service-acco,StorePaxDataAction
709610,33875264,2024-05-01 04:08:01,AB,2372,30,GAU,service-acco,StorePaxDataAction
709611,33875306,2024-05-01 04:09:01,AB,2372,30,GAU,service-acco,StorePaxDataAction


## CalculateWeightAndTrimAction           

In [9]:
calc_weight_and_trim_action_df = df[df.action_name == "CalculateWeightAndTrimAction"]
calc_weight_and_trim_action_df.head(4)

,id,creation_time,airline_code,flight_number,flight_date,departure_airport,user_name,action_name,header_line,entry_details
46743,34496931,2024-05-07 09:16:01,AB,2384,7,BBI,service-acco,CalculateWeightAndTrimAction,"2024-05-07 09:16:01,477 INFO [7698c4068c8ec2a...",START_WI weight : 45102.00 KG ...
46744,34496930,2024-05-07 09:16:01,AB,2384,7,BBI,service-acco,CalculateWeightAndTrimAction,"2024-05-07 09:16:01,477 INFO [7698c4068c8ec2a...",com.onesystem.lc2.common.dto.SingleAttributeDT...
46745,34496925,2024-05-07 09:16:01,AB,2562,7,CCU,service-acco,CalculateWeightAndTrimAction,"2024-05-07 09:16:01,217 INFO [ea50cb2e6df087f...",START_WI weight : 44170.00 KG ...
46746,34496924,2024-05-07 09:16:01,AB,2562,7,CCU,service-acco,CalculateWeightAndTrimAction,"2024-05-07 09:16:01,216 INFO [ea50cb2e6df087f...",com.onesystem.lc2.common.dto.SingleAttributeDT...


### parse random sample

In [10]:
def write_to_file(text: str, fileneame: str):
    with open(fileneame, "w") as file:
        file.write(text)

In [11]:
# CalculateWeightAndTrimAction random sample
random_index = random.randint(0, len(calc_weight_and_trim_action_df) - 1)
random_sample = str(calc_weight_and_trim_action_df.iloc[random_index].entry_details)
pd.set_option('display.max_colwidth', None)
random_sample

'com.onesystem.lc2.common.dto.SingleAttributeDTO[ id = NULL  deleted = false  legId = 543309 ]'

In [12]:
write_to_file(random_sample, "message.txt")

In [13]:
extracted_data = None

if "onesystem" in random_sample:
    extracted_data = random_sample
else:
    pattern = re.compile(r'(\w+(\s\w+)*)\s*:\s*([^\s]+)')

    # Find all matches
    matches = pattern.findall(random_sample)

    # Extracted keys and values
    extracted_data = {match[0].strip(): match[2] for match in matches}

print(extracted_data)

com.onesystem.lc2.common.dto.SingleAttributeDTO[ id = NULL  deleted = false  legId = 543309 ]


### parse all

In [14]:
# create columns
for idx, row in tqdm(calc_weight_and_trim_action_df.iterrows(), total=calc_weight_and_trim_action_df.shape[0], desc="Processing rows"):
    details = row["entry_details"]
    if "onesystem" in details:
        pass
    else:
        pattern = re.compile(r'(\w+(\s\w+)*)\s*:\s*([^\s]+)')
        matches = pattern.findall(details)
        extracted_data = {match[0].strip(): match[2] for match in matches}
        
        for key, value in extracted_data.items():
            if key not in calc_weight_and_trim_action_df.columns:
                calc_weight_and_trim_action_df[str(key)] = np.nan
            calc_weight_and_trim_action_df.at[idx, key] = value

Processing rows:   0%|          | 0/231990 [00:00<?, ?it/s]C:\Users\janmu\AppData\Local\Temp\ipykernel_35620\1792536218.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calc_weight_and_trim_action_df[str(key)] = np.nan
C:\Users\janmu\AppData\Local\Temp\ipykernel_35620\1792536218.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '45102.00' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  calc_weight_and_trim_action_df.at[idx, key] = value
C:\Users\janmu\AppData\Local\Temp\ipykernel_35620\1792536218.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [15]:
calc_weight_and_trim_action_df

,id,creation_time,airline_code,flight_number,flight_date,departure_airport,user_name,action_name,header_line,entry_details,...,TAIL_TIPPING_INDEX_EXCEEDED,FWD_MOVABLE_PAX,AFT_MOVABLE_PAX,INDEX_OUT_OF_BALANCE,LOAD_TO_AFT,LOAD_TO_FWD,ESTIMATED_TRAFFIC_LOAD,ESTIMATED_ZFW,DELTA_ZFW,ZFW_TOLERANCE_EXCEEDED
46743,34496931,2024-05-07 09:16:01,AB,2384,7,BBI,service-acco,CalculateWeightAndTrimAction,"2024-05-07 09:16:01,477 INFO [7698c4068c8ec2ad] [AB2384 /07 BBI | service-acco | CalculateWeightAndTrimAction | | Thread-63646 (ActiveMQ-client-global-threads)] Saved:",START_WI weight : 45102.00 KG START_WI index : 37.23 \r\n DO_WI weight : 45601.00 KG DO_WI index : 35.98 \r\n PAX_WI weight : 13540.00 KG PAX_WI index : 1.43 \r\n TOTAL_DEADLOAD_WI weight : 47415.00 KG TOTAL_DEADLOAD_WI index : 47.95 \r\n TOTAL_LOAD_WI : 1814.00 KG TOTAL_TRAFFIC_LOAD : 15354.00 KG \r\n FUEL_INDEX : 4.65 \r\n AZFW : 60955.00 KG ATOW : 71631.00 KG \r\n ALAW : 67568.00 KG ATXW : 71829.00 KG \r\n AFT_LIMIT_ZFW : 68.35 FWD_LIMIT_ZFW : 24.85 \r\n AFT_LIMIT_TOW : 77.89 FWD_LIMIT_TOW : 29.40 \r\n AFT_LIMIT_LAW : NULL FWD_LIMIT_LAW : NULL \r\n LIZFW : 49.37 LITOW : 54.02 \r\n LILAW : 54.83 MAC_AT_ZFW : 21.64 \r\n MAC_AT_TOW : 22.86 MAC_AT_LAW : 23.29 \r\n DEADLOAD_MAC : 21.42 \r\n UNDERLOAD : 1390.00 KG \r\n LIMITING_WEIGHT : LIMITING_WEIGHT_LAW \r\n ALLOWED TOW : 73021.00 KG ALLOWED ZFW : 60955.00 KG \r\n ALLOWED LAW : 67568.00 KG ALLOWED TXW : 71829.00 KG \r\n STABTO : NULL \r\n OPTIMAL_TRIM : 56.45 \r\n IDEAL_ADDITIONAL_LOAD_AFT : 459.51 KG IDEAL_ADDITIONAL_LOAD_FWD : -459.51 KG \r\n TAIL_TIPPING_WI weight : 60963.39 KG TAIL_TIPPING_WI index : 100.18 \r\n TAIL_TIPPING_INDEX_EXCEEDED : false \r\n FWD_MOVABLE_PAX : NULL AFT_MOVABLE_PAX : NULL \r\n INDEX_OUT_OF_BALANCE : NULL \r\n LOAD_TO_AFT : NULL LOAD_TO_FWD : NULL \r\n ESTIMATED_TRAFFIC_LOAD : 15354.00 KG ESTIMATED_ZFW : 60955.00 KG \r\n DELTA_ZFW : -245.00 KG ZFW_TOLERANCE_EXCEEDED : NULL,...,false,NULL,NULL,NULL,NULL,NULL,15354.00,60955.00,-245.00,NULL
46744,34496930,2024-05-07 09:16:01,AB,2384,7,BBI,service-acco,CalculateWeightAndTrimAction,"2024-05-07 09:16:01,477 INFO [7698c4068c8ec2ad] [AB2384 /07 BBI | service-acco | CalculateWeightAndTrimAction | | Thread-63646 (ActiveMQ-client-global-threads)] Received:",com.onesystem.lc2.common.dto.SingleAttributeDTO[ id = NULL deleted = false legId = 545343 ],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46745,34496925,2024-05-07 09:16:01,AB,2562,7,CCU,service-acco,CalculateWeightAndTrimAction,"2024-05-07 09:16:01,217 INFO [ea50cb2e6df087f1] [AB2562 /07 CCU | service-acco | CalculateWeightAndTrimAction | | Thread-63644 (ActiveMQ-client-global-threads)] Saved:",START_WI weight : 44170.00 KG START_WI index : 39.79 \r\n DO_WI weight : 44669.00 KG DO_WI index : 41.22 \r\n PAX_WI weight : 6485.00 KG PAX_WI index : 7.38 \r\n TOTAL_DEADLOAD_WI weight : 45174.00 KG TOTAL_DEADLOAD_WI index : 44.25 \r\n TOTAL_LOAD_WI : 505.00 KG TOTAL_TRAFFIC_LOAD : 6990.00 KG \r\n FUEL_INDEX : 6.22 \r\n AZFW : 51659.00 KG ATOW : 61047.00 KG \r\n ALAW : 55103.00 KG ATXW : 61245.00 KG \r\n AFT_LIMIT_ZFW : 55.25 FWD_LIMIT_ZFW : 27.05 \r\n AFT_LIMIT_TOW : 63.79 FWD_LIMIT_TOW : 21.32 \r\n AFT_LIMIT_LAW : NULL FWD_LIMIT_LAW : NULL \r\n LIZFW : 51.63 LITOW : 57.85 \r\n LILAW : 52.63 MAC_AT_ZFW : 22.91 \r\n MAC_AT_TOW : 24.76 MAC_AT_LAW : 23.14 \r\n DEADLOAD_MAC : 19.65 \r\n UNDERLOAD : 13855.00 KG \r\n LIMITING_WEIGHT : LIMITING_WEIGHT_LAW \r\n ALLOWED TOW : 74902.00 KG ALLOWED ZFW : 51659.00 KG \r\n ALLOWED LAW : 55103.00 KG ALLOWED TXW : 61245.00 KG \r\n STABTO : NULL \r\n OPTIMAL_TRIM : 43.35 \r\n IDEAL_ADDITIONAL_LOAD_AFT : -537.73 KG IDEAL_ADDITIONAL_LOAD_FWD : 537.73 KG \r\n TAIL_TIPPING_WI weight : 59070.00 KG TAIL_TIPPING_WI index : 77.84 \r\n TAIL_TIPPING_INDEX_EXCEEDED : false \r\n FWD_MOVABLE_PAX : NULL AFT_MOVABLE_PAX : NULL \r\n INDEX_OUT_OF_BALANCE : NULL \r\n LOAD_TO_AFT : NULL LOAD_TO_FWD : NULL \r\n ESTIMATED_TRAFFIC_LOAD : 6990.00 KG ESTIMATED_ZFW : 51659.00 KG \r\n DELTA_ZFW : -1441.0

In [16]:
calc_weight_and_trim_action_df.columns

Index(['id', 'creation_time', 'airline_code', 'flight_number', 'flight_date',
       'departure_airport', 'user_name', 'action_name', 'header_line',
       'entry_details', 'START_WI weight', 'START_WI index', 'DO_WI weight',
       'DO_WI index', 'PAX_WI weight', 'PAX_WI index',
       'TOTAL_DEADLOAD_WI weight', 'TOTAL_DEADLOAD_WI index', 'TOTAL_LOAD_WI',
       'TOTAL_TRAFFIC_LOAD', 'FUEL_INDEX', 'AZFW', 'ATOW', 'ALAW', 'ATXW',
       'AFT_LIMIT_ZFW', 'FWD_LIMIT_ZFW', 'AFT_LIMIT_TOW', 'FWD_LIMIT_TOW',
       'AFT_LIMIT_LAW', 'FWD_LIMIT_LAW', 'LIZFW', 'LITOW', 'LILAW',
       'MAC_AT_ZFW', 'MAC_AT_TOW', 'MAC_AT_LAW', 'DEADLOAD_MAC', 'UNDERLOAD',
       'LIMITING_WEIGHT', 'ALLOWED TOW', 'ALLOWED ZFW', 'ALLOWED LAW',
       'ALLOWED TXW', 'STABTO', 'OPTIMAL_TRIM', 'IDEAL_ADDITIONAL_LOAD_AFT',
       'IDEAL_ADDITIONAL_LOAD_FWD', 'TAIL_TIPPING_WI weight',
       'TAIL_TIPPING_WI index', 'TAIL_TIPPING_INDEX_EXCEEDED',
       'FWD_MOVABLE_PAX', 'AFT_MOVABLE_PAX', 'INDEX_OUT_OF_BALANCE',
  

In [17]:
calc_weight_and_trim_action_df

,id,creation_time,airline_code,flight_number,flight_date,departure_airport,user_name,action_name,header_line,entry_details,...,TAIL_TIPPING_INDEX_EXCEEDED,FWD_MOVABLE_PAX,AFT_MOVABLE_PAX,INDEX_OUT_OF_BALANCE,LOAD_TO_AFT,LOAD_TO_FWD,ESTIMATED_TRAFFIC_LOAD,ESTIMATED_ZFW,DELTA_ZFW,ZFW_TOLERANCE_EXCEEDED
46743,34496931,2024-05-07 09:16:01,AB,2384,7,BBI,service-acco,CalculateWeightAndTrimAction,"2024-05-07 09:16:01,477 INFO [7698c4068c8ec2ad] [AB2384 /07 BBI | service-acco | CalculateWeightAndTrimAction | | Thread-63646 (ActiveMQ-client-global-threads)] Saved:",START_WI weight : 45102.00 KG START_WI index : 37.23 \r\n DO_WI weight : 45601.00 KG DO_WI index : 35.98 \r\n PAX_WI weight : 13540.00 KG PAX_WI index : 1.43 \r\n TOTAL_DEADLOAD_WI weight : 47415.00 KG TOTAL_DEADLOAD_WI index : 47.95 \r\n TOTAL_LOAD_WI : 1814.00 KG TOTAL_TRAFFIC_LOAD : 15354.00 KG \r\n FUEL_INDEX : 4.65 \r\n AZFW : 60955.00 KG ATOW : 71631.00 KG \r\n ALAW : 67568.00 KG ATXW : 71829.00 KG \r\n AFT_LIMIT_ZFW : 68.35 FWD_LIMIT_ZFW : 24.85 \r\n AFT_LIMIT_TOW : 77.89 FWD_LIMIT_TOW : 29.40 \r\n AFT_LIMIT_LAW : NULL FWD_LIMIT_LAW : NULL \r\n LIZFW : 49.37 LITOW : 54.02 \r\n LILAW : 54.83 MAC_AT_ZFW : 21.64 \r\n MAC_AT_TOW : 22.86 MAC_AT_LAW : 23.29 \r\n DEADLOAD_MAC : 21.42 \r\n UNDERLOAD : 1390.00 KG \r\n LIMITING_WEIGHT : LIMITING_WEIGHT_LAW \r\n ALLOWED TOW : 73021.00 KG ALLOWED ZFW : 60955.00 KG \r\n ALLOWED LAW : 67568.00 KG ALLOWED TXW : 71829.00 KG \r\n STABTO : NULL \r\n OPTIMAL_TRIM : 56.45 \r\n IDEAL_ADDITIONAL_LOAD_AFT : 459.51 KG IDEAL_ADDITIONAL_LOAD_FWD : -459.51 KG \r\n TAIL_TIPPING_WI weight : 60963.39 KG TAIL_TIPPING_WI index : 100.18 \r\n TAIL_TIPPING_INDEX_EXCEEDED : false \r\n FWD_MOVABLE_PAX : NULL AFT_MOVABLE_PAX : NULL \r\n INDEX_OUT_OF_BALANCE : NULL \r\n LOAD_TO_AFT : NULL LOAD_TO_FWD : NULL \r\n ESTIMATED_TRAFFIC_LOAD : 15354.00 KG ESTIMATED_ZFW : 60955.00 KG \r\n DELTA_ZFW : -245.00 KG ZFW_TOLERANCE_EXCEEDED : NULL,...,false,NULL,NULL,NULL,NULL,NULL,15354.00,60955.00,-245.00,NULL
46744,34496930,2024-05-07 09:16:01,AB,2384,7,BBI,service-acco,CalculateWeightAndTrimAction,"2024-05-07 09:16:01,477 INFO [7698c4068c8ec2ad] [AB2384 /07 BBI | service-acco | CalculateWeightAndTrimAction | | Thread-63646 (ActiveMQ-client-global-threads)] Received:",com.onesystem.lc2.common.dto.SingleAttributeDTO[ id = NULL deleted = false legId = 545343 ],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46745,34496925,2024-05-07 09:16:01,AB,2562,7,CCU,service-acco,CalculateWeightAndTrimAction,"2024-05-07 09:16:01,217 INFO [ea50cb2e6df087f1] [AB2562 /07 CCU | service-acco | CalculateWeightAndTrimAction | | Thread-63644 (ActiveMQ-client-global-threads)] Saved:",START_WI weight : 44170.00 KG START_WI index : 39.79 \r\n DO_WI weight : 44669.00 KG DO_WI index : 41.22 \r\n PAX_WI weight : 6485.00 KG PAX_WI index : 7.38 \r\n TOTAL_DEADLOAD_WI weight : 45174.00 KG TOTAL_DEADLOAD_WI index : 44.25 \r\n TOTAL_LOAD_WI : 505.00 KG TOTAL_TRAFFIC_LOAD : 6990.00 KG \r\n FUEL_INDEX : 6.22 \r\n AZFW : 51659.00 KG ATOW : 61047.00 KG \r\n ALAW : 55103.00 KG ATXW : 61245.00 KG \r\n AFT_LIMIT_ZFW : 55.25 FWD_LIMIT_ZFW : 27.05 \r\n AFT_LIMIT_TOW : 63.79 FWD_LIMIT_TOW : 21.32 \r\n AFT_LIMIT_LAW : NULL FWD_LIMIT_LAW : NULL \r\n LIZFW : 51.63 LITOW : 57.85 \r\n LILAW : 52.63 MAC_AT_ZFW : 22.91 \r\n MAC_AT_TOW : 24.76 MAC_AT_LAW : 23.14 \r\n DEADLOAD_MAC : 19.65 \r\n UNDERLOAD : 13855.00 KG \r\n LIMITING_WEIGHT : LIMITING_WEIGHT_LAW \r\n ALLOWED TOW : 74902.00 KG ALLOWED ZFW : 51659.00 KG \r\n ALLOWED LAW : 55103.00 KG ALLOWED TXW : 61245.00 KG \r\n STABTO : NULL \r\n OPTIMAL_TRIM : 43.35 \r\n IDEAL_ADDITIONAL_LOAD_AFT : -537.73 KG IDEAL_ADDITIONAL_LOAD_FWD : 537.73 KG \r\n TAIL_TIPPING_WI weight : 59070.00 KG TAIL_TIPPING_WI index : 77.84 \r\n TAIL_TIPPING_INDEX_EXCEEDED : false \r\n FWD_MOVABLE_PAX : NULL AFT_MOVABLE_PAX : NULL \r\n INDEX_OUT_OF_BALANCE : NULL \r\n LOAD_TO_AFT : NULL LOAD_TO_FWD : NULL \r\n ESTIMATED_TRAFFIC_LOAD : 6990.00 KG ESTIMATED_ZFW : 51659.00 KG \r\n DELTA_ZFW : -1441.0

In [27]:
calc_weight_and_trim_action_df = calc_weight_and_trim_action_df.drop(columns=['creation_time', 'airline_code', 'flight_number', 'flight_date',
       'departure_airport', 'user_name', 'action_name', 'header_line',
       'entry_details'])

In [28]:
parsed_df = pd.merge(standard_df, calc_weight_and_trim_action_df, on='id', how='left')
parsed_df.columns

Index(['id', 'creation_time', 'airline_code', 'flight_number', 'flight_date',
       'departure_airport', 'user_name', 'action_name', 'START_WI weight',
       'START_WI index', 'DO_WI weight', 'DO_WI index', 'PAX_WI weight',
       'PAX_WI index', 'TOTAL_DEADLOAD_WI weight', 'TOTAL_DEADLOAD_WI index',
       'TOTAL_LOAD_WI', 'TOTAL_TRAFFIC_LOAD', 'FUEL_INDEX', 'AZFW', 'ATOW',
       'ALAW', 'ATXW', 'AFT_LIMIT_ZFW', 'FWD_LIMIT_ZFW', 'AFT_LIMIT_TOW',
       'FWD_LIMIT_TOW', 'AFT_LIMIT_LAW', 'FWD_LIMIT_LAW', 'LIZFW', 'LITOW',
       'LILAW', 'MAC_AT_ZFW', 'MAC_AT_TOW', 'MAC_AT_LAW', 'DEADLOAD_MAC',
       'UNDERLOAD', 'LIMITING_WEIGHT', 'ALLOWED TOW', 'ALLOWED ZFW',
       'ALLOWED LAW', 'ALLOWED TXW', 'STABTO', 'OPTIMAL_TRIM',
       'IDEAL_ADDITIONAL_LOAD_AFT', 'IDEAL_ADDITIONAL_LOAD_FWD',
       'TAIL_TIPPING_WI weight', 'TAIL_TIPPING_WI index',
       'TAIL_TIPPING_INDEX_EXCEEDED', 'FWD_MOVABLE_PAX', 'AFT_MOVABLE_PAX',
       'INDEX_OUT_OF_BALANCE', 'LOAD_TO_AFT', 'LOAD_TO_FWD',
    

In [30]:
parsed_df.head()

,id,creation_time,airline_code,flight_number,flight_date,departure_airport,user_name,action_name,START_WI weight,START_WI index,...,TAIL_TIPPING_INDEX_EXCEEDED,FWD_MOVABLE_PAX,AFT_MOVABLE_PAX,INDEX_OUT_OF_BALANCE,LOAD_TO_AFT,LOAD_TO_FWD,ESTIMATED_TRAFFIC_LOAD,ESTIMATED_ZFW,DELTA_ZFW,ZFW_TOLERANCE_EXCEEDED
0,33902381,2024-05-01 09:14:42,AB,2359,1,BLR,service-acco,ASMMsgProcessor,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,33910427,2024-05-01 10:28:12,AB,2104,2,BLR,service-acco,ASMMsgProcessor,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,33910380,2024-05-01 10:28:11,AB,2109,2,BLR,service-acco,ASMMsgProcessor,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,33909790,2024-05-01 10:28:12,AB,2110,2,BOM,service-acco,ASMMsgProcessor,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,33902706,2024-05-01 09:14:41,AB,2127,2,BOM,service-acco,ASMMsgProcessor,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# StorePaxDataAction                     